In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data_train = pd.read_csv('./data/application_train.csv')
data_test = pd.read_csv('./data/application_test.csv')

In [3]:
np.mean(data_train.TARGET==0), np.mean(data_train.TARGET==1)

(0.91927118054313506, 0.080728819456864956)

In [4]:
data_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
data_train_1 = data_train.loc[data_train.loc[data_train.index,'TARGET']==0]
data_train_0 = data_train.loc[data_train.loc[data_train.index,'TARGET']==1]

data_train_1.shape, data_train_0.shape

((282686, 122), (24825, 122))

In [6]:
data_train_1 = data_train_1[:len(data_train_0)]

In [7]:
data_train_1.shape, data_train_0.shape

((24825, 122), (24825, 122))

In [8]:
data_train = pd.concat((data_train_0,data_train_1)).sort_index()
data_train.shape, data_test.shape

((49650, 122), (48744, 121))

In [9]:
def label_encoder(df):
    assert sum(df.dtypes == 'object') >= 1, 'dataframe not have a values of type object'
    #se saca la lista de columnas de tipo object
    list_columns_object = df.dtypes[df.dtypes == 'object'].index
    for index in list_columns_object:
        #se cambian los tipos a category para poder aplicar el .cat.codes de pandas
        df.loc[:,index] = df.loc[:,index].astype('category').cat.codes
        
    return df

In [10]:
#se codifican los datos tanto de entrenamiento y prueba
data_train = label_encoder(data_train)
data_test = label_encoder(data_test)

In [11]:
data_train = data_train.fillna(value=0)
data_test = data_test.fillna(value=0)

In [12]:
X_train = data_train[[col for col in  data_train.columns if col!='TARGET']]
y_train = data_train.TARGET
X_test = data_test

print('Dimensiones')
print('X train: ',X_train.shape)
print('y train: ',y_train.shape)
print('X test: ',X_test.shape)

Dimensiones
X train:  (49650, 121)
y train:  (49650,)
X test:  (48744, 121)


In [17]:
from sklearn.ensemble import RandomForestClassifier

RFC = RandomForestClassifier(n_jobs=3, n_estimators=15)
RFC.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=15, n_jobs=3,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [18]:
RFC.score(X_train, y_train)

0.99808660624370593

In [19]:
predict = RFC.predict_proba(X_test)

In [20]:
predict

array([[ 0.93333333,  0.06666667],
       [ 0.8       ,  0.2       ],
       [ 0.86666667,  0.13333333],
       ..., 
       [ 0.06666667,  0.93333333],
       [ 0.2       ,  0.8       ],
       [ 0.        ,  1.        ]])

In [21]:
predict_proba = []
for i in predict:
    if i[0]>=[1]:
        predict_proba.append(i[0])
    else: predict_proba.append(i[1])

In [22]:
predict_proba

[0.066666666666666666,
 0.20000000000000001,
 0.13333333333333333,
 0.066666666666666666,
 0.40000000000000002,
 0.20000000000000001,
 0.066666666666666666,
 0.20000000000000001,
 0.066666666666666666,
 0.20000000000000001,
 0.066666666666666666,
 0.20000000000000001,
 0.46666666666666667,
 0.13333333333333333,
 0.20000000000000001,
 0.26666666666666666,
 0.20000000000000001,
 0.26666666666666666,
 0.26666666666666666,
 0.20000000000000001,
 0.13333333333333333,
 0.20000000000000001,
 0.13333333333333333,
 0.13333333333333333,
 1.0,
 0.13333333333333333,
 0.066666666666666666,
 0.066666666666666666,
 0.26666666666666666,
 0.26666666666666666,
 1.0,
 1.0,
 0.13333333333333333,
 0.13333333333333333,
 0.26666666666666666,
 0.33333333333333331,
 0.26666666666666666,
 0.40000000000000002,
 0.26666666666666666,
 0.20000000000000001,
 0.26666666666666666,
 0.066666666666666666,
 0.26666666666666666,
 0.26666666666666666,
 0.13333333333333333,
 0.40000000000000002,
 0.066666666666666666,
 0.20

In [52]:
result = pd.DataFrame(data=predict_proba, index=data_test.SK_ID_CURR, columns=['TARGET'])

In [53]:
result.index.name = 'SK_ID_CURR'

In [54]:
result.to_csv('./results/submission_test_10.csv')

In [55]:
result.head()

,TARGET
SK_ID_CURR,
100001,0.266667
100005,0.066667
100013,0.133333
100028,1.000000
100038,0.133333


In [56]:
from sklearn.naive_bayes import GaussianNB

GNB = GaussianNB()
GNB.fit(X_train,y_train)

GaussianNB(priors=None)

In [57]:
GNB.predict_proba(X_test)

array([[  9.78898945e-001,   2.11010545e-002],
       [  9.73757294e-001,   2.62427062e-002],
       [  9.97354169e-001,   2.64583069e-003],
       ..., 
       [  3.79790820e-305,   1.00000000e+000],
       [  5.89712377e-305,   1.00000000e+000],
       [  3.42877905e-305,   1.00000000e+000]])

In [59]:
predict_proba = []
for i in predict:
    if i[0]>=[1]:
        predict_proba.append(i[0])
    else: predict_proba.append(i[1])

In [60]:
result = pd.DataFrame(data=predict_proba, index=data_test.SK_ID_CURR, columns=['TARGET'])

In [61]:
result.index.name = 'SK_ID_CURR'

In [62]:
result.to_csv('./results/submission_test_10.csv')

In [38]:
X_train.shape

(49650, 121)

In [39]:
from sklearn.neighbors import KNeighborsClassifier

KNC = KNeighborsClassifier()
KNC.fit(X_train[:24000],y_train[:24000])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [41]:
print(KNC.score(X_train[:24000], y_train[:24000]))
KNC.score(X_train[24000:], y_train[24000:])

0.922083333333


0.11606237816764133

In [34]:
predict = KNC.predict_proba(X_test)

In [35]:
predict_proba = []
for i in predict:
    if i[0]>=[1]:
        predict_proba.append(i[0])
    else: predict_proba.append(i[1])

In [36]:
predict_proba

[1.0,
 0.20000000000000001,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.40000000000000002,
 1.0,
 0.20000000000000001,
 0.20000000000000001,
 1.0,
 0.40000000000000002,
 0.40000000000000002,
 0.20000000000000001,
 1.0,
 1.0,
 1.0,
 0.20000000000000001,
 0.20000000000000001,
 1.0,
 1.0,
 0.20000000000000001,
 1.0,
 0.20000000000000001,
 1.0,
 0.20000000000000001,
 1.0,
 0.40000000000000002,
 0.20000000000000001,
 0.40000000000000002,
 1.0,
 1.0,
 1.0,
 1.0,
 0.40000000000000002,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.20000000000000001,
 1.0,
 1.0,
 0.40000000000000002,
 1.0,
 1.0,
 0.20000000000000001,
 1.0,
 1.0,
 1.0,
 1.0,
 0.20000000000000001,
 1.0,
 1.0,
 0.40000000000000002,
 1.0,
 1.0,
 0.20000000000000001,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.40000000000000002,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.20000000000000001,
 1.0,
 1.0,
 0.20000000000000001,
 0.20000000000000001,
 0.20000000000000001,
 1.0,
 0.20000000000000001,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,


In [37]:
result = pd.DataFrame(data=predict_proba, index=data_test.SK_ID_CURR, columns=['TARGET'])
result.index.name = 'SK_ID_CURR'
#result.to_csv('./results/submission_test_11.csv')

## PROBANDO NUEVAS COSAS

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data_train = pd.read_csv('./data/application_train.csv')
data_test = pd.read_csv('./data/application_test.csv')

In [3]:
def label_encoder(df):
    assert sum(df.dtypes == 'object') >= 1, 'dataframe not have a values of type object'
    #se saca la lista de columnas de tipo object
    list_columns_object = df.dtypes[df.dtypes == 'object'].index
    for index in list_columns_object:
        #se cambian los tipos a category para poder aplicar el .cat.codes de pandas
        df.loc[:,index] = df.loc[:,index].astype('category').cat.codes
        
    return df

In [4]:
data_train = label_encoder(data_train)
data_test = label_encoder(data_test)

In [5]:
data_train_1 = data_train[data_train.loc[data_train.index,'TARGET']==1]
data_train_0 = data_train[data_train.loc[data_train.index,'TARGET']==0][:len(data_train_1)]

data_train_1.shape, data_train_0.shape

((24825, 122), (24825, 122))

In [6]:
data_train = pd.concat((data_train_0,data_train_1)).sort_index()
data_train.shape, data_test.shape

((49650, 122), (48744, 121))

In [7]:
data_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,0,1,0,1,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,0,0,0,0,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,1,1,1,1,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,0,0,0,1,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,0,1,0,1,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
sum(data_train.TARGET==1),sum(data_train.TARGET==0) 

(24825, 24825)

In [9]:
y_train = data_train.TARGET

In [10]:
matriz_corr = data_train.corr()

In [36]:
col_best = matriz_corr.loc[:,((matriz_corr > 0.7).sum() > 1)==True].columns

In [37]:
col_best

Index(['SK_ID_CURR', 'TARGET', 'CNT_CHILDREN', 'AMT_CREDIT', 'AMT_ANNUITY',
       'AMT_GOODS_PRICE', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT',
       'REGION_RATING_CLIENT_W_CITY', 'REG_REGION_NOT_WORK_REGION',
       'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_WORK_CITY',
       'LIVE_CITY_NOT_WORK_CITY', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG',
       'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG',
       'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG',
       'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG',
       'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG', 'APARTMENTS_MODE',
       'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BUILD_MODE',
       'COMMONAREA_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE', 'FLOORSMAX_MODE',
       'FLOORSMIN_MODE', 'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE',
       'LIVINGAREA_MODE', 'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE',
       'APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'YEARS_BEGIN

In [38]:
X_train = data_train[[col for col in  col_best if col!='TARGET']].fillna(0)
X_test = data_test[[col for col in  col_best if col!='TARGET']].fillna(0)

X_train.shape, y_train.shape, X_test.shape

((49650, 62), (49650,), (48744, 62))

In [14]:
from sklearn.naive_bayes import GaussianNB

GNB = GaussianNB()
GNB.fit(X_train,y_train)

GaussianNB(priors=None)

In [33]:
GNB.score(X_train,y_train)

0.94594159113796572

In [15]:
predict = GNB.predict_proba(X_test)

In [18]:
predict[:,1]

array([ 0.58804641,  0.57474197,  0.55809717, ...,  0.59115289,
        0.59344256,  0.59255641])

In [37]:
predict_proba = []
for i in predict:
    if i[0]>=i[1]:
        predict_proba.append(i[1])
    else: predict_proba.append(i[0])

In [38]:
predict_proba

[0.12457383467727356,
 0.1233047076012881,
 0.015403264075310074,
 0.0032837675191048944,
 0.17369016221407443,
 0.077660105180688568,
 0.15999752032205108,
 0.17472826486037057,
 0.18093244053603738,
 0.11742322023981035,
 0.01514868850696155,
 0.146316868241535,
 0.17803993635746171,
 0.13125017428765329,
 0.15392128128882993,
 0.19109881739334142,
 0.12532290695482345,
 0.020630903347484048,
 0.22819586091650393,
 0.099624831454211915,
 0.21330447572427588,
 0.083522843867360586,
 0.018747593919787822,
 0.041619242857169017,
 0.19728771152537741,
 0.15267115621329955,
 0.017828600760683214,
 0.10333071425464332,
 0.15373268141197671,
 0.058446631812807771,
 0.15889982207375744,
 0.1634156303408342,
 0.23014125414743772,
 0.18867785357175654,
 0.10485484711401846,
 0.14453413059374334,
 5.00755478923109e-07,
 0.20499200920791558,
 0.062515774285172715,
 0.23774733492781577,
 0.1314943772659706,
 0.017067158596421231,
 0.16836807021410041,
 0.09698368779220029,
 0.09465387285265521,
 

In [48]:
result = pd.DataFrame(data=predict[:,1], index=data_test.SK_ID_CURR, columns=['TARGET'])
result.index.name = 'SK_ID_CURR'
result.to_csv('./results/submission_test_18.csv')

In [24]:
from sklearn.svm import SVC

clf = SVC(probability=True)

In [25]:
clf.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [26]:
predict = clf.predict_proba(X_test)

In [27]:
predict[:,1]

array([ 0.57490829,  0.4262069 ,  0.5       , ...,  0.40603921,
        0.54266193,  0.59665751])

In [28]:
#correlación mayor a 0.8
result = pd.DataFrame(data=predict[:,1], index=data_test.SK_ID_CURR, columns=['TARGET'])
result.index.name = 'SK_ID_CURR'
result.to_csv('./results/submission_test_20.csv')

In [29]:
from lightgbm import LGBMClassifier

In [39]:
clf = LGBMClassifier(
        objective='binary',
        n_estimators=300,
        num_leaves=15,
        colsample_bytree=.8,
        subsample=.8,
        max_depth=7,
        reg_alpha=.1,
        reg_lambda=.1,
        min_split_gain=.01
    )

In [40]:
clf.fit(X_train, y_train, eval_metric='auc')

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.8,
        learning_rate=0.1, max_depth=7, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.01, n_estimators=300,
        n_jobs=-1, num_leaves=15, objective='binary', random_state=None,
        reg_alpha=0.1, reg_lambda=0.1, silent=True, subsample=0.8,
        subsample_for_bin=200000, subsample_freq=1)

In [41]:
clf.predict_proba(X_test)[:,1]

array([ 0.05037475,  0.07221925,  0.06537194, ...,  0.99989664,
        0.99992533,  0.99995143])

In [43]:
#correlacion mayor a 0.7
result = pd.DataFrame(data=predict[:,1], index=data_test.SK_ID_CURR, columns=['TARGET'])
result.index.name = 'SK_ID_CURR'
result.to_csv('./results/submission_test_21.csv')

In [72]:
predict_proba = []
for i in predict:
    if i[0]>=i[1]:
        predict_proba.append(i[0])
    else: predict_proba.append(i[1])

In [73]:
predict_proba

[0.96941666745377419,
 0.95672368506315375,
 0.99713632246140504,
 0.96065253614086576,
 0.88573369841933491,
 0.97769492229030341,
 0.92897789515862828,
 0.51218458352020457,
 0.83512025304366799,
 0.50316736916247218,
 0.97502621646192855,
 0.89717260995625769,
 0.76437782381717068,
 0.9793541939188718,
 0.67235997574899886,
 0.82624852672000015,
 0.98019200471437284,
 0.94162436568758145,
 0.67580679211736827,
 0.98553325655023682,
 0.64435318089477478,
 0.9904162166778252,
 0.96779990378999325,
 0.77923637545957425,
 0.84615304982524164,
 0.88718324162113094,
 0.87087043517248597,
 0.98759233488991527,
 0.933729651029646,
 0.97965745165345886,
 0.86639159182397818,
 0.86797186669449811,
 0.52993747747762399,
 0.82324340774431815,
 0.95151090020958839,
 0.54954114137599785,
 0.99834892841102618,
 0.79467287266256115,
 0.96832521339046407,
 0.57955986216787903,
 0.94858989908088609,
 0.97812226255478718,
 0.86942878064187001,
 0.97375302999559121,
 0.98271098864111361,
 0.98819110128

In [50]:
result = pd.DataFrame(data=predict, index=data_test.SK_ID_CURR, columns=['TARGET'])
result.index.name = 'SK_ID_CURR'
result.to_csv('./results/submission_test_16.csv')